In [ ]:
import imbrium
import pandas as pd
print(imbrium.__version__)

In [ ]:
from imbrium.predictors.univarstandard import *

In [ ]:
data = pd.read_csv('example_dataset/CaliforniaHousing.csv')
data = data['target']

## UnivarMultiStep

In [ ]:
predictor1 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = '')
predictor1.create_mlp(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (25, 'relu'), 'layer1': (5,'relu'), 'layer2': (25, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data.tail(5))
predictor1.model_blueprint()

In [ ]:
predictor2 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'normalize')
predictor2.create_rnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (20, 'relu'), 'layer1': (25,'relu'), 'layer2': (5, 'relu')})
predictor2.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor2.show_performance()
predictor2.predict(data.tail(5))
predictor2.model_blueprint()

In [ ]:
predictor3 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'standard')
predictor3.create_lstm(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (20, 'relu'), 'layer1': (25,'relu'), 'layer2': (5, 'relu')})
predictor3.fit_model(epochs = 10, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor3.show_performance()
predictor3.predict(data.tail(5))
predictor3.model_blueprint()

In [ ]:
predictor4 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'minmax')
predictor4.create_gru(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (20, 'relu'), 'layer1': (25,'relu'), 'layer2': (5, 'relu')})
predictor4.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor4.show_performance()
predictor4.predict(data.tail(5))
predictor4.model_blueprint()

In [ ]:
predictor5 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = '')
predictor5.create_cnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu')})
predictor5.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor5.show_performance()
predictor5.predict(data.tail(5))
predictor5.model_blueprint()

In [ ]:
predictor6 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'standard')
predictor6.create_birnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (15, 'relu'), 'layer1': (5, 'relu')})
predictor6.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor6.show_performance()
predictor6.predict(data.tail(5))
predictor6.model_blueprint()

In [ ]:
predictor7 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'minmax')
predictor7.create_bilstm(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (15, 'relu'), 'layer1': (5, 'relu')})
predictor7.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor7.show_performance()
predictor7.predict(data.tail(5))
predictor7.model_blueprint()

In [ ]:
predictor8 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'maxabs')
predictor8.create_bigru(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (15, 'relu'), 'layer1': (5, 'relu')})
predictor8.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor8.show_performance()
predictor8.predict(data.tail(5))
predictor8.model_blueprint()

In [ ]:
predictor9 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'normalize')
predictor9.create_encdec_rnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (50, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (1, 'relu')})
predictor9.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor9.show_performance()
predictor9.predict(data.tail(5))
predictor9.model_blueprint()

In [ ]:
predictor10 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'normalize')
predictor10.create_encdec_lstm(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (50, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (1, 'relu')})
predictor10.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor10.show_performance()
predictor10.predict(data.tail(5))
predictor10.model_blueprint()

In [ ]:
predictor11 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'normalize')
predictor11.create_encdec_cnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (1, 'relu')})
predictor11.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor11.show_performance()
predictor11.predict(data.tail(5))
predictor11.model_blueprint()

In [ ]:
predictor12 = BasicMultStepUniVar(steps_past =  5, steps_future = 10, data = data, scale = 'minmax')
predictor12.create_encdec_gru(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (50, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (1, 'relu')})
predictor12.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor12.show_performance()
predictor12.predict(data.tail(5))
predictor12.model_blueprint()

In [ ]:
data.tail(5)

In [ ]:
predictor12.save_model()

In [ ]:
load = BasicMultStepUniVar(steps_past =  5, steps_future = 10)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.load_model('univarstandard')

In [ ]:
load.predict(data.tail(5))

In [ ]:
from imbrium.predictors.univarhybrid import *

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'maxabs')
predictor1.create_cnnrnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 10, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'normalize')
predictor1.create_cnnrnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'standard')
predictor1.create_cnnlstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'minmax')
predictor1.create_cnngru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'normalize')
predictor1.create_cnnbirnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'normalize')
predictor1.create_cnnbilstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data, scale = 'normalize')
predictor1.create_cnnbigru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (25, 'relu'), 'layer4': (10, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data.tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1.save_model()

In [ ]:
load = HybridMultStepUniVar(sub_seq = 2, steps_past =  10, steps_future = 5)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.load_model('univarhybrid')

In [ ]:
load.predict(data.tail(10))

## MultiVarMultiStep

In [ ]:
from imbrium.predictors.multivarstandard import *

In [ ]:
data = pd.read_csv('example_dataset/CaliforniaHousing.csv')

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = '')
#predictor1.get_X_input
predictor1.create_mlp(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5,'relu'), 'layer2': (5, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_rnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5,'relu'), 'layer2': (5, 'relu')})
predictor1.fit_model(epochs = 10, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'standard')
predictor1.create_lstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5,'relu'), 'layer2': (5, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
from imbrium.predictors.univarhybrid import HybridMultStepUniVar

predictor = HybridMultStepUniVar(sub_seq = 2, steps_past = 10, steps_future = 5,
                                  data = data, scale = 'minmax')

predictor.create_cnnbilstm(optimizer= 'adam', loss= 'mean_squared_error',
                            metrics = 'mean_squared_error'
                           )

predictor.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)

predictor.predict(data.tail(10))

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'minmax')
predictor1.create_cnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (64, 1, 'relu'), 'layer1': (32, 1, 'relu'), 'layer2': (2), 'layer3': (50, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'maxabs')
predictor1.create_gru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5,'relu'), 'layer2': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_birnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_bilstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor1 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_bigru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor1.model_blueprint()

In [ ]:
predictor9 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor9.create_encdec_rnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (100, 'relu')})
predictor9.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor9.show_performance()
predictor9.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor9.model_blueprint()

In [ ]:
predictor10 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor10.create_encdec_lstm(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (100, 'relu')})
predictor10.fit_model(epochs = 10, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor10.show_performance()
predictor10.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor10.model_blueprint()

In [ ]:
predictor11 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor11.create_encdec_cnn(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (10, 'relu')})
predictor11.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor11.show_performance()
predictor11.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor11.model_blueprint()

In [ ]:
predictor12 = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor12.create_encdec_gru(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (5, 'relu'), 'layer1': (5, 'relu'), 'layer2': (5, 'relu'), 'layer3': (100, 'relu')})
predictor12.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor12.show_performance()
predictor12.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))
predictor12.model_blueprint()

In [ ]:
predictor1.save_model()

In [ ]:
load = BasicMultStepMultVar(steps_past =  5, steps_future = 10, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.load_model('multivarstandard')

In [ ]:
load.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(5))

In [ ]:
from imbrium.predictors.multivarhybrid import HybridMultStepMultVar

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
#predictor1.get_X_input
predictor1.create_cnnrnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 3, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'])
predictor1.create_cnnlstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_cnngru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 10, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_cnnbirnn(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_cnnbilstm(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 2, show_progress= 1, validation_split = 0.20, batch_size = 10)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1 = HybridMultStepMultVar(sub_seq = 2, steps_past = 10, steps_future = 5, data = data,features = ['target','target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')
predictor1.create_cnnbigru(optimizer= 'adam', loss= 'mean_squared_error', metrics = 'mean_squared_error', layer_config = {'layer0': (8, 1, 'relu'), 'layer1': (4, 1, 'relu'), 'layer2': (2), 'layer3': (5, 'relu'), 'layer4': (5, 'relu')})
predictor1.fit_model(epochs = 7, show_progress= 1, validation_split = 0.20, batch_size = 10, monitor='val_loss', patience=1, min_delta = 0, verbose = 1)
predictor1.show_performance()
predictor1.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))
predictor1.model_blueprint()

In [ ]:
predictor1.save_model()

In [ ]:
load = HybridMultStepMultVar(sub_seq = 2, steps_past =  10, steps_future = 5, data = data, features = ['target', 'target', 'HouseAge', 'AveRooms', 'AveBedrms'], scale = 'normalize')

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.load_model('multivarhybrid')

In [ ]:
load.predict(data[['target', 'HouseAge', 'AveRooms', 'AveBedrms']].tail(10))